# Guia do Projeto - Tech Challenge Fase 4 - Parte 05

## Análise de Cenas e Geração de Resumo

Até agora, processamos o vídeo frame a frame. Porém, para um humano, um vídeo é feito de **histórias** ou **cenas**, não de frames isolados. Uma cena é um segmento contínuo que acontece no mesmo lugar e tempo.

Neste notebook, vamos:
1.  **Detectar Cenas Automaticamente**: Usar histogramas de cor para identificar quando a câmera muda de ângulo ou local (cortes).
2.  **Agregar Dados**: Em vez de dizer "Frame 1: Feliz, Frame 2: Feliz...", diremos "Cena 1 (0s-5s): Emoção Dominante = Feliz".
3.  **Gerar Relatório**: Criar uma tabela resumo com contagem de pessoas, emoções e ações por cena.

### Como funciona a Detecção de Cenas?

Usamos uma técnica simples e eficaz baseada em **Histogramas de Cor**.
*   Calculamos a distribuição de cores (histograma) do frame atual.
*   Comparamos com o histograma do frame anterior.
*   Se a diferença for muito grande (maior que um limiar), assumimos que houve um **corte** de cena.

In [5]:
import cv2
import sys
import os
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Dict, Deque, Any, Generator
from collections import deque, Counter
from ultralytics import YOLO
from deepface import DeepFace
import logging

os.environ["YOLO_VERBOSE"] = "False"
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### 1. Classes de Suporte (Consolidadas)

Para este notebook funcionar sozinho, incluímos todas as classes criadas nas partes anteriores, além das novas classes de Cena.

In [6]:
# --- Estruturas de Dados ---

@dataclass
class BoundingBox:
    x: int; y: int; width: int; height: int

@dataclass
class FaceDetection:
    bounding_box: BoundingBox; confidence: float

@dataclass
class EmotionAnalysis:
    emotion: str; confidence: float

@dataclass
class ActivityDetection:
    activity: str; confidence: float; track_id: Optional[int] = None

@dataclass
class Scene:
    scene_id: int
    start_frame: int
    end_frame: int
    start_time: float
    end_time: float
    @property
    def duration_seconds(self) -> float: return self.end_time - self.start_time

@dataclass
class SceneResult:
    scene: Scene
    unique_faces: int = 0
    dominant_emotions: Dict[str, int] = field(default_factory=dict)
    dominant_actions: Dict[str, int] = field(default_factory=dict)

# --- Detectores (Versões Simplificadas para Demonstração) ---
# Em produção, usaríamos as versões completas dos notebooks anteriores

class FaceDetector:
    def __init__(self, model_path: str = "yolo11n-pose.pt"):
        self.model = YOLO(model_path)
    
    def detect(self, frame) -> List[FaceDetection]:
        # Detecção simplificada
        results = self.model(frame, verbose=False)
        faces = []
        if results[0].boxes:
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                faces.append(FaceDetection(BoundingBox(x1, y1, x2-x1, y2-y1), float(box.conf)))
        return faces

class EmotionAnalyzer:
    def analyze(self, frame, faces) -> List[EmotionAnalysis]:
        # Simulação para demonstração rápida (DeepFace é lento sem GPU)
        # Em produção, descomentar o código real do notebook 03
        return [EmotionAnalysis("neutral", 0.9) for _ in faces]

class ActivityDetector:
    def __init__(self, model_path: str = "yolo11n-pose.pt"):
        self.model = YOLO(model_path)
        
    def detect(self, frame) -> List[ActivityDetection]:
        # Detecção simplificada com tracking
        results = self.model.track(frame, persist=True, verbose=False, tracker="bytetrack.yaml")
        activities = []
        if results[0].boxes and results[0].boxes.id is not None:
            ids = results[0].boxes.id.cpu().numpy().astype(int)
            for i, track_id in enumerate(ids):
                # Lógica simplificada: assumindo "standing" para teste
                activities.append(ActivityDetection("standing", 0.9, track_id))
        return activities

# --- Detector de Cenas (O Novo Protagonista) ---

class SceneDetector:
    def __init__(self, threshold=0.5):
        self.threshold = threshold
    
    def detect_scenes(self, video_path: str) -> List[Scene]:
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        scenes = []
        scene_start = 0
        scene_id = 1
        
        ret, prev = cap.read()
        if not ret: return []
        
        # Calcular histograma do primeiro frame
        prev_hist = cv2.calcHist([prev], [0], None, [256], [0, 256])
        cv2.normalize(prev_hist, prev_hist, 0, 1, cv2.NORM_MINMAX)
        
        frame_num = 1
        while True:
            ret, curr = cap.read()
            if not ret: break
            
            # Calcular histograma do frame atual
            curr_hist = cv2.calcHist([curr], [0], None, [256], [0, 256])
            cv2.normalize(curr_hist, curr_hist, 0, 1, cv2.NORM_MINMAX)
            
            # Comparar histogramas (Correlação)
            # 1.0 = idêntico, < 0.5 = muito diferente (corte)
            score = cv2.compareHist(prev_hist, curr_hist, cv2.HISTCMP_CORREL)
            
            if score < self.threshold:
                # Corte detectado! Salvar cena anterior
                scenes.append(Scene(scene_id, scene_start, frame_num-1, scene_start/fps, (frame_num-1)/fps))
                scene_id += 1
                scene_start = frame_num
            
            prev_hist = curr_hist
            frame_num += 1
            
        # Salvar a última cena
        scenes.append(Scene(scene_id, scene_start, frame_num-1, scene_start/fps, (frame_num-1)/fps))
        cap.release()
        return scenes

### 2. O Analisador de Vídeo (Orquestrador)

Esta classe gerencia tudo: detecta as cenas, e depois percorre cada cena extraindo informações (amostrando frames para não ficar lento).

In [7]:
class VideoSceneAnalyzer:
    def __init__(self, video_path, scene_threshold=0.6, sample_rate=5):
        self.video_path = video_path
        self.sample_rate = sample_rate # Processar 1 a cada N frames
        self.scene_detector = SceneDetector(scene_threshold)
        self.face_detector = FaceDetector()
        self.emotion_analyzer = EmotionAnalyzer()
        self.activity_detector = ActivityDetector()
        
    def analyze(self) -> List[SceneResult]:
        print("1. Detectando cenas (pode demorar um pouco)...")
        scenes = self.scene_detector.detect_scenes(self.video_path)
        print(f"   -> {len(scenes)} cenas detectadas.")
        
        results = []
        cap = cv2.VideoCapture(self.video_path)
        
        for scene in scenes:
            print(f"2. Processando Cena {scene.scene_id} ({scene.duration_seconds:.1f}s)...")
            cap.set(cv2.CAP_PROP_POS_FRAMES, scene.start_frame)
            
            emotions_cnt = Counter()
            actions_cnt = Counter()
            unique_faces = set()
            
            curr = scene.start_frame
            while curr <= scene.end_frame:
                ret, frame = cap.read()
                if not ret: break
                
                # Amostragem para performance
                if (curr - scene.start_frame) % self.sample_rate == 0:
                    # Detectar Atividades e IDs únicos
                    acts = self.activity_detector.detect(frame)
                    for a in acts:
                        actions_cnt[a.activity] += 1
                        if a.track_id: unique_faces.add(a.track_id)
                    
                    # Detectar Emoções
                    faces = self.face_detector.detect(frame)
                    ems = self.emotion_analyzer.analyze(frame, faces)
                    for e in ems: emotions_cnt[e.emotion] += 1
                
                curr += 1
            
            results.append(SceneResult(
                scene, 
                len(unique_faces), 
                dict(emotions_cnt.most_common(2)), # Top 2 emoções
                dict(actions_cnt.most_common(2))   # Top 2 ações
            ))
            
        cap.release()
        return results

### 3. Execução e Relatório

Vamos rodar o analisador e formatar a saída em uma tabela legível.

In [8]:
video_path = "meu_video.mp4"

try:
    # Threshold 0.6 geralmente funciona bem para cortes secos
    analyzer = VideoSceneAnalyzer(video_path, scene_threshold=0.6, sample_rate=10)
    scene_results = analyzer.analyze()

    # Gerar DataFrame para visualização
    data = []
    for res in scene_results:
        scene = res.scene
        emotions_str = ", ".join([f"{k}({v})" for k, v in res.dominant_emotions.items()])
        actions_str = ", ".join([f"{k}({v})" for k, v in res.dominant_actions.items()])
        
        data.append({
            "Cena": scene.scene_id,
            "Início (s)": f"{scene.start_time:.1f}",
            "Duração (s)": f"{scene.duration_seconds:.1f}",
            "Pessoas Únicas": res.unique_faces,
            "Emoções Dominantes": emotions_str,
            "Ações Dominantes": actions_str
        })

    df = pd.DataFrame(data)
    print("\n=== Relatório de Análise de Cenas ===")
    print(df.to_string(index=False))

except Exception as e:
    print(f"Erro: {e}")

1. Detectando cenas (pode demorar um pouco)...
   -> 18 cenas detectadas.
2. Processando Cena 1 (6.0s)...
2. Processando Cena 2 (6.0s)...
2. Processando Cena 3 (6.0s)...
2. Processando Cena 4 (6.0s)...
2. Processando Cena 5 (6.0s)...
2. Processando Cena 6 (6.0s)...
2. Processando Cena 7 (6.0s)...
2. Processando Cena 8 (6.0s)...
2. Processando Cena 9 (6.0s)...
2. Processando Cena 10 (3.0s)...
2. Processando Cena 11 (4.0s)...
2. Processando Cena 12 (6.0s)...
2. Processando Cena 13 (12.0s)...
2. Processando Cena 14 (1.0s)...
2. Processando Cena 15 (12.0s)...
2. Processando Cena 16 (6.0s)...
2. Processando Cena 17 (12.0s)...
2. Processando Cena 18 (0.8s)...

=== Relatório de Análise de Cenas ===
 Cena Início (s) Duração (s)  Pessoas Únicas Emoções Dominantes Ações Dominantes
    1        0.0         6.0               4        neutral(74)     standing(72)
    2        6.0         6.0               1        neutral(18)     standing(18)
    3       12.0         6.0               0            

### Conclusão da Parte 05

Neste notebook, elevamos o nível da análise de frames individuais para **cenas completas**.

**O que foi construído:**
*   **Detector de Cenas:** Implementação robusta usando histogramas de cor para identificar cortes.
*   **Agregador de Dados:** Lógica para consolidar emoções e atividades dominantes por cena.
*   **Relatório Estruturado:** Geração de um DataFrame com o resumo narrativo do vídeo.

**Próximo Passo:**
No notebook final (**06-final_pipeline.ipynb**), vamos integrar todos os componentes (Vídeo, Face, Emoção, Atividade, Cena) em uma única classe `VideoPipeline` para processar qualquer vídeo de ponta a ponta.